In [1]:
import numpy as np

In [2]:
from ipynb.fs.full.wspFFA import wspFirefly
from ipynb.fs.full.wspCS import wspCuckooSearch
from ipynb.fs.full.wspKH import wspKrillHerd
from ipynb.fs.full.wspEHO import wspElephantHerding
from ipynb.fs.full.wspABC import wspArtificialBeeColony

In [ ]:
from ipynb.fs.full.wspShannonEvaluation import wspShannonEvaluation
from ipynb.fs.full.wspTsallisEvaluation import wspTsallisEvaluation

In [3]:
def wspGrayHistogram(img):
    hist, _ = np.histogram(img.ravel(),256,[0,255])

    hist = hist/np.sum(hist)

    return hist

In [ ]:
def apply_threshold(img, thresh):
    row, col =  img.shape
    
    colors = [*range(0,255, round(255/len(thresh)))]
    colors.append(255)
    #print(colors, thresh)

    img_thres = np.zeros((row,col))

    for i in range(0, row):
        for j in range(0, col):
            pixel = img[i,j]
            color = -1

            for k in range(0, len(thresh)):
                if pixel < thresh[k]:
                    color = colors[k]
                    break
                 
            if color == -1:
                color = colors[len(colors)-1]

            img_thres[i,j] = color

    return img_thres

In [ ]:
def run_firefly(hist, dimension, entropy, q):
    n = 50
    d = dimension
    gama = 1
    alpha = .97
    beta = 1
    maxGeneration = 100

    best_thresholds = wspFirefly(n,d,gama,alpha,beta,maxGeneration, hist, entropy, q)

    return best_thresholds

In [ ]:
def run_cuckoo_search(hist, dimension, entropy, q):
    n = 40
    d = dimension
    pa = 0.5
    maxGeneration = 100

    best_thresholds = wspCuckooSearch(n,d,pa, maxGeneration, hist, entropy, q)

    return best_thresholds

In [ ]:
def run_krill_herd(hist, dimension, entropy, q):
    n = 40
    d = dimension
    maxGeneration = 100

    best_thresholds = wspKrillHerd(n, d, maxGeneration, hist, entropy, q)

    return best_thresholds 

In [ ]:
def run_elephant_herding(hist, dimension, entropy, q):
    n = 200
    nkE = 2
    nClan = 5
    alpha = 0.5
    beta = 0.1
    d = dimension
    maxGeneration = 100

    best_thresholds = wspElephantHerding(n, d, nkE, nClan, alpha, beta, maxGeneration, hist, entropy, q)

    return best_thresholds

In [ ]:
def run_artificial_bee_colony(hist, dimension, entropy, q):
    n = 20
    d = dimension
    maxGeneration = 100
    
    best_thresholds = wspArtificialBeeColony(n, d, maxGeneration, hist, entropy, q)
    
    return best_thresholds

In [1]:
def switch(alg):
    if alg == 'FFA': return run_firefly
    elif alg == 'CS': return run_cuckoo_search
    elif alg == 'KH': return run_krill_herd
    elif alg == 'EHO': return run_elephant_herding
    elif alg == 'ABC': return run_artificial_bee_colony

In [ ]:
def wspMultithreshold(img, algorithm, dimension, q):
    hist = wspGrayHistogram(img)
    
    if q == 1:
        entropy = wspShannonEvaluation
    else:
        entropy = wspTsallisEvaluation
    
    chosen_algorithm = switch(algorithm) 
    best_thresholds = chosen_algorithm(hist, dimension, entropy, q)

    img_thres = apply_threshold(img, best_thresholds)

    return hist, best_thresholds, img_thres